<a href="https://colab.research.google.com/github/mushiboy/Deep-Learning-FastAI-/blob/master/Plant_Kaggle_Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
  !curl -s https://course.fast.ai/setup/colab | bash

In [0]:
from pathlib import Path

In [0]:
!pip install -q kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mushiboy","key":"7dd3e678ce1b62c7e5ac4ed7156e6844"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 ~/.kaggle/kaggle.json


In [0]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!kaggle -v


Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 8.1MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 4.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=3d0a478a19909cb0bdabe9a09788b2562049e820d1f57592d72e01c2b3b914bb
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle
Kaggle API 1.5.6


In [0]:
!kaggle competitions download -c plant-pathology-2020-fgvc7 -p /content/gdrive/My\ Drive/plant

 99% 769M/779M [00:11<00:00, 77.3MB/s]
100% 779M/779M [00:11<00:00, 73.0MB/s]


In [0]:
!unzip -q -n '/content/gdrive/My Drive/plant/plant-pathology-2020-fgvc7.zip' -d '/content/gdrive/My Drive/plant'

In [0]:
from fastai import *
from fastai.vision import *
from fastai.metrics import error_rate


In [0]:
import pandas as pd

In [0]:
#Image path
path = Path("/content/gdrive/My Drive/plant")

In [0]:
path.ls()

[PosixPath('/content/gdrive/My Drive/plant/plant-pathology-2020-fgvc7.zip'),
 PosixPath('/content/gdrive/My Drive/plant/sample_submission.csv'),
 PosixPath('/content/gdrive/My Drive/plant/train.csv'),
 PosixPath('/content/gdrive/My Drive/plant/images'),
 PosixPath('/content/gdrive/My Drive/plant/test.csv')]

In [0]:
tfms = tfms = get_transforms()


In [0]:
df.head()

NameError: ignored

In [0]:
df = pd.read_csv("/content/gdrive/My Drive/plant/train.csv")

In [0]:
df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [0]:
df['Max'] = df[['healthy','multiple_diseases', 'rust', 'scab']].idxmax(axis=1)

In [0]:
df.drop(['healthy','multiple_diseases', 'rust', 'scab'], axis = 1, inplace = True)
df.head()

,image_id,Max
0,Train_0,scab
1,Train_1,multiple_diseases
2,Train_2,healthy
3,Train_3,rust
4,Train_4,healthy


In [0]:
data = (ImageList.from_df(df, path, folder = "images", suffix = ".jpg")
      .split_by_rand_pct()
      .label_from_df()
      .transform(tfms, size = 128)
      .databunch()).normalize()


Exception: ignored

In [0]:
data.classes

In [0]:
data.show_batch(3, fig_size = (7,6))

In [0]:
learn = cnn_learner(data, models.resnet34, metrics = error_rate)

In [0]:
learn.fit_one_cycle(1)

In [0]:
learn.save("Try1")

In [0]:
learn.unfreeze()

In [0]:
learn.fit_one_cycle(1)

In [0]:
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(4, max_lr=1.5e-3)

In [0]:
df2 = pd.DataFrame(learn.get_preds()[0])

In [0]:
df2.head()

In [0]:
train_preds, train_ground_truths = learn.get_preds(ds_type=DatasetType.Valid)


In [0]:
train_paths = list(map(lambda path: str(path), learn.data.valid_ds.items))
train_fnames = list(map(lambda path: path.split('/')[-1][:-4], train_paths))
train_fnames_and_ground_truths = list(zip(train_fnames, train_ground_truths))

In [0]:
train_paths